# Задача классификации

Sklearn - библиотека машинного обучения на языке Python. В ней реализовано практически все необходимое для машинного обучения.

+ разделение выборки на тестовую и обучающую
+ основные метрики
+ кроссвалидация
+ линейная регрессия и ее модификация
+ задачи классификации
+ кластеризация
+ различные методы работы с признаками

Познакомимся с частью ее фишек сначала на тренировочном примере, а потом на примере прогнозирования ледового затора

### **0. Импорт необходимых пакетов**

In [29]:
import pandas as pd
import numpy as np

# нам понадобится отрисовка графиков

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### **1. Загрузка данных**


В качестве тренировочного датасета возьмем различные метеоданные Австралии и будем предсказывать, будет ли завтра дождь).


In [30]:
data = pd.read_csv('weatherAUS.csv', sep = ',')

In [31]:
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


Посмотрим, какие у нас признаки, есть ли пропуски.

In [32]:
data.shape

(142193, 24)

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        141556 non-null  float64
 3   MaxTemp        141871 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81350 non-null   float64
 6   Sunshine       74377 non-null   float64
 7   WindGustDir    132863 non-null  object 
 8   WindGustSpeed  132923 non-null  float64
 9   WindDir9am     132180 non-null  object 
 10  WindDir3pm     138415 non-null  object 
 11  WindSpeed9am   140845 non-null  float64
 12  WindSpeed3pm   139563 non-null  float64
 13  Humidity9am    140419 non-null  float64
 14  Humidity3pm    138583 non-null  float64
 15  Pressure9am    128179 non-null  float64
 16  Pressure3pm    128212 non-null  float64
 17  Cloud9am       88536 non-null

Описание принаков. https://www.kaggle.com/jsphyg/weather-dataset-rattle-package/data

+ **Location** The common name of the location of the weather station
+ **MinTemp** The minimum temperature in degrees celsius
+ **MaxTemp** The maximum temperature in degrees celsius
+ **Rainfall** The amount of rainfall recorded for the day in mm
+ **Evaporation** The so-called Class A pan evaporation (mm) in the 24 hours to 9am
+ **Sunshine** The number of hours of bright sunshine in the day.
+ **WindGustDir** The direction of the strongest wind gust in the 24 hours to midnight
+ **WindGustSpeed** The speed (km/h) of the strongest wind gust in the 24 hours to midnight
+ **WindDir9am** Direction of the wind at 9am
+ **WindDir3pm** Direction of the wind at 3pm
+ **WindSpeed9am** Wind speed (km/hr) averaged over 10 minutes prior to 9am
+ **WindSpeed3pm** Wind speed (km/hr) averaged over 10 minutes prior to 3pm
+ **Humidity9am** Humidity (percent) at 9am
+ **Humidity3pm** Humidity (percent) at 3pm
+ **Pressure9am** Atmospheric pressure (hpa) reduced to mean sea level at 9am
+ **Pressure3pm** Atmospheric pressure (hpa) reduced to mean sea level at 3pm
+ **Cloud9am** Fraction of sky obscured by cloud at 9am. This is measured in "oktas", which are a unit of eigths. It records how many eigths of the sky are obscured by cloud. A 0 measure indicates completely clear sky whilst an 8 indicates that it is completely overcast.
+ **Cloud3pm** Fraction of sky obscured by cloud (in "oktas": eighths) at 3pm. See Cload9am for a description of the values
+ **Temp9am** Temperature (degrees C) at 9am
+ **Temp3pm** Temperature (degrees C) at 3pm
+ **RainToday** Boolean: 1 if precipitation (mm) in the 24 hours to 9am exceeds 1mm, otherwise 0
+ **RISK_MM** The amount of next day rain in mm. Used to create response variable RainTomorrow. A kind of measure of the "risk".
+ **RainTomorrow** The target variable. Did it rain tomorrow?

**Посмотрим, на сбалансированность выборки**

In [34]:
data.RainTomorrow.value_counts()

No     110316
Yes     31877
Name: RainTomorrow, dtype: int64

Видим, что выборка не очень сбалансированна, но это не слишком критично. 

**Что будем делать с пропусками?**

Первое, что мы сделаем - посчитаем, сколько у нас пропусков у каждого из признаков

In [35]:
data.isnull().sum()

Date                 0
Location             0
MinTemp            637
MaxTemp            322
Rainfall          1406
Evaporation      60843
Sunshine         67816
WindGustDir       9330
WindGustSpeed     9270
WindDir9am       10013
WindDir3pm        3778
WindSpeed9am      1348
WindSpeed3pm      2630
Humidity9am       1774
Humidity3pm       3610
Pressure9am      14014
Pressure3pm      13981
Cloud9am         53657
Cloud3pm         57094
Temp9am            904
Temp3pm           2726
RainToday         1406
RISK_MM              0
RainTomorrow         0
dtype: int64

**Мы видим, что есть несколько признаков, у которых пропусков порядка половины. Тут ничего не поделаешь - придется удалять их**

Плюс нам не нужны в качестве признаков дата, локация и значение осадков на следующий день (их можно будет успользовать как целевую переменную при предсказании количества осадков на следующий день)

In [36]:
data.drop(labels = ['Date','Location','Evaporation',
                       'Sunshine','Cloud3pm','Cloud9am','RISK_MM'],axis = 1,inplace = True)

In [37]:
data.head()

,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,13.4,22.9,0.6,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8,No,No
1,7.4,25.1,0.0,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3,No,No
2,12.9,25.7,0.0,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2,No,No
3,9.2,28.0,0.0,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5,No,No
4,17.5,32.3,1.0,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7,No,No


**Выделим столбец с предсказаниями и таблицу с признаками**

Видим, что прогноз на завтра и за сегодня имеют тип объекта, а нам хотелось бы число. 0 или 1. Перекодируем

In [38]:
data['RainTomorrow'] = data['RainTomorrow'].map({'Yes': 1, 'No': 0}) 
data['RainToday'] = data['RainToday'].map({'Yes': 1, 'No': 0}) 
data.head()

,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,13.4,22.9,0.6,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8,0.0,0
1,7.4,25.1,0.0,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3,0.0,0
2,12.9,25.7,0.0,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2,0.0,0
3,9.2,28.0,0.0,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5,0.0,0
4,17.5,32.3,1.0,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7,0.0,0


In [39]:
data.isnull().sum()

MinTemp            637
MaxTemp            322
Rainfall          1406
WindGustDir       9330
WindGustSpeed     9270
WindDir9am       10013
WindDir3pm        3778
WindSpeed9am      1348
WindSpeed3pm      2630
Humidity9am       1774
Humidity3pm       3610
Pressure9am      14014
Pressure3pm      13981
Temp9am            904
Temp3pm           2726
RainToday         1406
RainTomorrow         0
dtype: int64

Остальных пропусков не очень много - безжалостно удаляем наблюдения с ними с помощью функции dropna

In [40]:
data.dropna(inplace = True)

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112925 entries, 0 to 142192
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   MinTemp        112925 non-null  float64
 1   MaxTemp        112925 non-null  float64
 2   Rainfall       112925 non-null  float64
 3   WindGustDir    112925 non-null  object 
 4   WindGustSpeed  112925 non-null  float64
 5   WindDir9am     112925 non-null  object 
 6   WindDir3pm     112925 non-null  object 
 7   WindSpeed9am   112925 non-null  float64
 8   WindSpeed3pm   112925 non-null  float64
 9   Humidity9am    112925 non-null  float64
 10  Humidity3pm    112925 non-null  float64
 11  Pressure9am    112925 non-null  float64
 12  Pressure3pm    112925 non-null  float64
 13  Temp9am        112925 non-null  float64
 14  Temp3pm        112925 non-null  float64
 15  RainToday      112925 non-null  float64
 16  RainTomorrow   112925 non-null  int64  
dtypes: float64(13), int64(1), obj

**У нас осталось три признака типа оbject. Что будем с ними делать?** 

Заменим каждый признак на несколько признаков с помощью функции get_dummies. Что она делает?

Если признак принимает три значения, она создает три новых признака - флаги каждого из значений

In [42]:
categorical = ['WindGustDir','WindDir9am','WindDir3pm']

data = pd.get_dummies(data,columns = categorical,drop_first=True)

In [43]:
data.head()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,...,0,0,0,0,0,0,0,0,1,0
1,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,...,0,0,0,0,0,0,0,0,0,1
2,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,...,0,0,0,0,0,0,0,0,0,1
3,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,...,0,0,0,0,0,0,0,0,0,0
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,...,0,1,0,0,0,0,0,0,0,0


In [44]:
data.shape

(112925, 59)

Теперь создаем таблицу признаков и вектор целевой переменной

In [45]:
y = data['RainTomorrow']
data.drop(['RainTomorrow'], axis=1, inplace=True)

In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112925 entries, 0 to 142192
Data columns (total 58 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   MinTemp          112925 non-null  float64
 1   MaxTemp          112925 non-null  float64
 2   Rainfall         112925 non-null  float64
 3   WindGustSpeed    112925 non-null  float64
 4   WindSpeed9am     112925 non-null  float64
 5   WindSpeed3pm     112925 non-null  float64
 6   Humidity9am      112925 non-null  float64
 7   Humidity3pm      112925 non-null  float64
 8   Pressure9am      112925 non-null  float64
 9   Pressure3pm      112925 non-null  float64
 10  Temp9am          112925 non-null  float64
 11  Temp3pm          112925 non-null  float64
 12  RainToday        112925 non-null  float64
 13  WindGustDir_ENE  112925 non-null  uint8  
 14  WindGustDir_ESE  112925 non-null  uint8  
 15  WindGustDir_N    112925 non-null  uint8  
 16  WindGustDir_NE   112925 non-null  uint

### **3. Построим самые первые и простые модели**

Данных очень много, поэтому попробуем работать только с 20% всех данных

Выделим 70% выборки (X_train, y_train) под обучение и 30% будут тестовой выборкой (X_test, y_test) с помощью функции **train_test_split**. Тестовая выборка никак не будет участвовать в настройке параметров моделей, на ней мы в конце, после этой настройки, оценим качество полученной модели. 

Построим решающее дерево и метод 1 ближайшего соседа (параметры пока подбирать не будем), а так же сравним результаты со случайным и негативным классификатором (дождя нет). 

In [47]:
from sklearn.model_selection import train_test_split

In [48]:
# загрузим функцию разделения на тестовую и обучающую
from sklearn.model_selection import train_test_split

X_learn, X_another, Y_learn, Y_another = train_test_split(data, y, test_size = 0.8, random_state = 42)
X_train, X_test, Y_train, Y_test = train_test_split(X_learn, Y_learn, test_size = 0.3, random_state = 42)

In [49]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

tree = DecisionTreeClassifier(max_depth=2, random_state=42)
neigh = KNeighborsClassifier(n_neighbors=1)

#Обучение модели - функция fit

tree.fit(X_train, Y_train)
neigh.fit(X_train, Y_train)

KNeighborsClassifier(n_neighbors=1)

Проверим качество построенных моделей.
В качестве метрик посмотрим на accuracy, precision, recall.

In [50]:
# получить предсказания обученной модели по новому набору данных - функция predict
tree_pred = tree.predict(X_test)
neigh_pred = neigh.predict(X_test)

In [51]:
from sklearn import metrics

In [52]:
from sklearn.metrics import accuracy_score

print('Real accuracy', 1 - Y_test.sum()/len(Y_test))
print('Tree', accuracy_score(Y_test, tree_pred))
print('Neighboors',accuracy_score(Y_test, neigh_pred))

Real accuracy 0.7854191263282172
Tree 0.8265938606847698
Neighboors 0.7888134592680047


Мы видим, что если мы просто запустим две простые модели, что качество уже будет лучше, чем если бы мы просто сказали, что дождя никогда не будет. Но у нас несбалансированная выбока

In [58]:
from sklearn.metrics import recall_score, precision_score, balanced_accuracy_score

In [ ]:
?balanced_accuracy_score

In [ ]:
from sklearn.metrics import recall_score, precision_score, balanced_accuracy_score
print('Decision Tree:')
print('Accuracy', accuracy_score(Y_test, tree_pred))
print('Balanced accuracy', balanced_accuracy_score(Y_test, tree_pred))
print('Precision', precision_score(Y_test, tree_pred, pos_label=1))
print('Recall', recall_score(Y_test, tree_pred))
print('1 NN:')
print('Accuracy', accuracy_score(Y_test, neigh_pred))
print('Balanced accuracy', balanced_accuracy_score(Y_test, tree_pred))
print('Precision', precision_score(Y_test, neigh_pred))
print('Recall', recall_score(Y_test, neigh_pred))


### **4. Настроим параметры модели**

### Дерево
Настроим оптимальные параметры для нашего дерева. Будем оптимизировать максимальную глубину дерева и максимальное используемое число признаков при каждом разбиении.
GridSearchCV - обход по сетке параметров. Для каждой уникальной пары значений параметров max_depth и max_features будет проведена 10-кратная кросс-валидация и выберется лучшее сочетание параметров.

В качестве метрики будем использовать balanced accuracy. 

Названия метрик можно посмотреть тут https://scikit-learn.org/stable/modules/model_evaluation.html

In [ ]:
?DecisionTreeClassifier

In [53]:
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold

In [54]:
%%time
tree_params = {
    'max_depth': range(1,10),
    'max_features': range(4,20)
}
kf = KFold(random_state = 42, shuffle = True, n_splits = 5)

tree_grid = GridSearchCV(tree, tree_params, cv=kf, n_jobs=-1, scoring='balanced_accuracy')
# смотрим, что у нас будет на нашей решетке на тренировочных данных
tree_grid.fit(X_train, Y_train)

print(tree_grid.best_params_, tree_grid.best_score_)

{'max_depth': 4, 'max_features': 10} 0.7030817431173848
Wall time: 18.4 s


In [55]:
tree_grid.best_params_

{'max_depth': 4, 'max_features': 10}

Обучим нашу модель на этих параметрах и посмотрим качество модели на тесте

In [59]:
tree_best = DecisionTreeClassifier(max_depth=4, max_features=10, random_state=42)

tree_best.fit(X_train, Y_train)
tree_best_pred = tree_best.predict(X_test)

print('Decision Tree:')
print('Accuracy', accuracy_score(Y_test, tree_best_pred))
print('Balanced accuracy', balanced_accuracy_score(Y_test, tree_best_pred))
print('Precision', precision_score(Y_test, tree_best_pred, pos_label=1))
print('Recall', recall_score(Y_test, tree_best_pred))


Decision Tree:
Accuracy 0.8209858323494688
Balanced accuracy 0.710088847673383
Precision 0.5957108816521048
Recall 0.515818431911967


In [60]:
from sklearn.metrics import classification_report

print(classification_report(Y_test,tree_best_pred))

              precision    recall  f1-score   support

           0       0.87      0.90      0.89      5322
           1       0.60      0.52      0.55      1454

    accuracy                           0.82      6776
   macro avg       0.73      0.71      0.72      6776
weighted avg       0.81      0.82      0.82      6776



### KNN 

Замечание!

На лекции обсуждалось, что метрические классификаторы очень чувствительны к масштабу признаков. Попробуем сначала признаки отмасштабировать

In [61]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
?MinMaxScaler

In [62]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

sc_X_train = scaler.fit_transform(X_train)
sc_X_test = scaler.transform(X_test)


In [63]:
neigh_sc = KNeighborsClassifier(n_neighbors=1)

neigh_sc.fit(sc_X_train, Y_train)
neigh_pred = neigh_sc.predict(sc_X_test)



In [ ]:
print('1 NN:')
print('Accuracy', accuracy_score(Y_test, neigh_pred))
print('Balanced accuracy', balanced_accuracy_score(Y_test, neigh_pred))
print('Precision', precision_score(Y_test, neigh_pred))
print('Recall', recall_score(Y_test, neigh_pred))
print(classification_report(Y_test,tree_best_pred))

Запускать подбор параметров по всем признакам - смертельный номер. Компьютер просто умрет.

Поэтому выберем 10 самых наилучших признаков

In [65]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif

In [67]:
best_f = SelectKBest(f_classif, k=10)
X_train_new = best_f.fit_transform(X_train, Y_train)
X_test_new = best_f.transform(X_test)

### Подберем для каждого масштабирования оптимальные параметры. Выберем наилучший вариант. Обучим модель и проверим ее качество на тренировочной выборке

количество соседей от 1 до 15, веса - равномерные и расстояние, метрика - евклидова и минковского

In [68]:
%%time
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler

scalers = [StandardScaler(), MaxAbsScaler(), MinMaxScaler()]

for scaler in scalers:
    #print(str(scaler))
    sc_X_train = scaler.fit_transform(X_train_new)
    
    grid_params = {
        'n_neighbors':  np.arange(1, 16, 2),
        'weights': ['uniform', 'distance'],
        'metric': ['minkowski', 'manhattan']
    }
    
    KNN = KNeighborsClassifier()
    kf = KFold(random_state = 42, shuffle = True, n_splits = 5)
    grid = GridSearchCV(KNN, grid_params, cv = kf, scoring = 'balanced_accuracy')
    grid.fit(sc_X_train, Y_train)
    
    print(scaler, grid.best_params_, grid.best_score_)

StandardScaler() {'metric': 'minkowski', 'n_neighbors': 7, 'weights': 'distance'} 0.7014685685846629
MaxAbsScaler() {'metric': 'minkowski', 'n_neighbors': 9, 'weights': 'uniform'} 0.7029251965604205
MinMaxScaler() {'metric': 'minkowski', 'n_neighbors': 11, 'weights': 'uniform'} 0.70674752039054
Wall time: 3min 24s


In [69]:
scaler = MinMaxScaler()

sc_X_train = scaler.fit_transform(X_train_new)
sc_X_test = scaler.transform(X_test_new)

KNN_best = KNeighborsClassifier(metric = 'minkowski', n_neighbors = 11, weights = 'uniform')

KNN_best.fit(sc_X_train, Y_train)
KNN_best_pred = KNN_best.predict(sc_X_test)



In [70]:
print('Best KNN:')
print('Accuracy', accuracy_score(Y_test, KNN_best_pred))
print('Balanced accuracy', balanced_accuracy_score(Y_test, KNN_best_pred))
print('Precision', precision_score(Y_test, KNN_best_pred, pos_label=1))
print('Recall', recall_score(Y_test, KNN_best_pred))
print(classification_report(Y_test,tree_best_pred))

Best KNN:
Accuracy 0.8479929161747344
Balanced accuracy 0.7092867219044046
Precision 0.7274678111587983
Recall 0.46629986244841815
              precision    recall  f1-score   support

           0       0.87      0.90      0.89      5322
           1       0.60      0.52      0.55      1454

    accuracy                           0.82      6776
   macro avg       0.73      0.71      0.72      6776
weighted avg       0.81      0.82      0.82      6776



In [ ]:
Accuracy 0.7590023612750886
Balanced accuracy 0.6261425801492546
Precision 0.4323507180650038
Recall 0.3933975240715268